In [70]:
import json
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver

from supabase import create_client, Client

import os

from dotenv import load_dotenv

In [71]:
load_dotenv()
supabase_password = os.getenv('supabase_password')
project_url = os.getenv('project_url')
project_api = os.getenv('project_api')

In [72]:
driver = webdriver.Chrome()

In [73]:
whoscored_url = 'https://www.whoscored.com/Matches/1832494/Live/International-European-Championship-2024-Spain-Germany'

In [74]:
driver.get(whoscored_url)

In [75]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [76]:
element = soup.select_one('script:-soup-contains("matchCentreData")')

In [77]:
matchdict = json.loads(element.text.split("matchCentreData: ") [1].split(',\n') [0])

In [78]:
matchdict.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [79]:
matchdict['events'][55]

{'id': 2699283899.0,
 'eventId': 35,
 'minute': 3,
 'second': 20,
 'teamId': 336,
 'playerId': 31772,
 'x': 67.0,
 'y': 28.6,
 'expandedMinute': 3,
 'period': {'value': 1, 'displayName': 'FirstHalf'},
 'type': {'value': 4, 'displayName': 'Foul'},
 'outcomeType': {'value': 0, 'displayName': 'Unsuccessful'},
 'qualifiers': [{'type': {'value': 13, 'displayName': 'Foul'}},
  {'type': {'value': 233, 'displayName': 'OppositeRelatedEvent'},
   'value': '32'},
  {'type': {'value': 286, 'displayName': 'Offensive'}},
  {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'},
  {'type': {'value': 178, 'displayName': 'StandingSave'}}],
 'satisfiedEventsTypes': [64],
 'isTouch': False}

In [80]:
match_events = matchdict['events']

In [81]:
df = pd.DataFrame(match_events)

In [82]:
df.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,endY,relatedEventId,relatedPlayerId,blockedX,blockedY,goalMouthZ,goalMouthY,isShot,cardType,isGoal
0,2.699284e+09,2,0,0.0,338,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.699284e+09,2,0,0.0,336,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.699284e+09,3,0,0.0,336,50.0,50.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.699284e+09,4,0,2.0,336,44.6,49.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,55.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.699284e+09,3,0,4.0,338,22.8,37.4,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 12, 'displayName': 'Clearance'}",...,43.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df.dropna(subset='playerId', inplace=True)

In [84]:
#Find the null values and change it into None
df = df.where(pd.notnull(df), None)

In [85]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'blockedX', 'blockedY',
       'goalMouthZ', 'goalMouthY', 'isShot', 'cardType', 'isGoal'],
      dtype='object')

In [86]:
#Rename all the columns
df = df.rename(
    {
        'eventId': 'event_id',
        'expandedMinute': 'expanded_minute',
        'outcomeType': 'outcome_type',
        'isTouch': 'is_touch',
        'playerId': 'player_id',
        'teamId': 'team_id',
        'endX': 'end_x',
        'endY': 'end_y',
        'blockedX': 'blocked_x',
        'blockedY': 'blocked_y',
        'goalMouthZ': 'goal_mouth_z',
        'goalMouthY': 'goal_mouth_y',
        'isShot': 'is_shot',
        'cardType': 'card_type',
        'isGoal': 'is_goal'
    },
    axis=1
)

In [87]:
#Drop the dictionary in those columns
df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])

In [88]:
#Add is_goal column in the df
if 'is_goal' not in df.columns:
    print('missing goals')
    df['is_goal'] = False

In [89]:
len(df.index)

1962

In [90]:
#Exclude OffsideGiven data
df = df[~(df['type_display_name'] == 'OffsideGiven')]

In [91]:
df.columns

Index(['id', 'event_id', 'minute', 'second', 'team_id', 'x', 'y',
       'expanded_minute', 'period', 'type', 'outcome_type', 'qualifiers',
       'satisfiedEventsTypes', 'is_touch', 'player_id', 'end_x', 'end_y',
       'relatedEventId', 'relatedPlayerId', 'blocked_x', 'blocked_y',
       'goal_mouth_z', 'goal_mouth_y', 'is_shot', 'card_type', 'is_goal',
       'period_display_name', 'type_display_name',
       'outcome_type_display_name'],
      dtype='object')

In [92]:
df['type_display_name'].head()

2            Pass
3            Pass
4       Clearance
5    Dispossessed
6          Tackle
Name: type_display_name, dtype: object

In [93]:
df.drop(columns=['period', 'type', 'outcome_type'], inplace=True)

In [94]:
df = df[[
    'id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
    'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
    'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
    'period_display_name'
]]

In [95]:
df.dtypes

id                           float64
event_id                       int64
minute                         int64
second                       float64
team_id                        int64
player_id                    float64
x                            float64
y                            float64
end_x                        float64
end_y                        float64
qualifiers                    object
is_touch                        bool
blocked_x                    float64
blocked_y                    float64
goal_mouth_z                 float64
goal_mouth_y                 float64
is_shot                       object
card_type                     object
is_goal                       object
type_display_name             object
outcome_type_display_name     object
period_display_name           object
dtype: object

In [96]:
#Casting appropriate types for variables
df[['id', 'event_id', 'minute', 'team_id', 'player_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id']].astype(np.int64)

df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)

df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)

In [97]:
df['is_goal'] = df['is_goal'].fillna(False)
df['is_shot'] = df['is_shot'].fillna(False)

In [98]:
#Replace all NaN values in columns with the data types np.float64 or np.float32 with None
for column in df.columns:
    if df[column].dtype == np.float64 or df[column].dtype == np.float32:
        df[column] = np.where(
            np.isnan(df[column]),
            None,
            df[column]
        )

In [99]:
df.iloc[0].to_dict()

{'id': 2699283635,
 'event_id': 3,
 'minute': 0,
 'second': 0.0,
 'team_id': 336,
 'player_id': 326413,
 'x': 50.0,
 'y': 50.0,
 'end_x': 44.5,
 'end_y': 51.0,
 'qualifiers': [{'type': {'value': 212, 'displayName': 'Length'},
   'value': '5.8'},
  {'type': {'value': 213, 'displayName': 'Angle'}, 'value': '3.02'},
  {'type': {'value': 140, 'displayName': 'PassEndX'}, 'value': '44.5'},
  {'type': {'value': 178, 'displayName': 'StandingSave'}},
  {'type': {'value': 141, 'displayName': 'PassEndY'}, 'value': '51.0'},
  {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Back'}],
 'is_touch': True,
 'blocked_x': None,
 'blocked_y': None,
 'goal_mouth_z': None,
 'goal_mouth_y': None,
 'is_shot': False,
 'card_type': False,
 'is_goal': False,
 'type_display_name': 'Pass',
 'outcome_type_display_name': 'Successful',
 'period_display_name': 'FirstHalf'}

In [100]:
class MatchEvent(BaseModel):
    id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    player_id: int
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [101]:
#This code attempts to create MatchEvent objects for each row of a 
#df by converting each row into a dictionary of arguments.

for x in df.to_dict(orient='records'):
    try:
        MatchEvent(**x).dict()
    except Exception as e:
        print(e)
        break


C:\Users\Duy Tran\AppData\Local\Temp\ipykernel_27640\3288689499.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  MatchEvent(**x).dict()


In [102]:
def insert_match_events(df, supabase):
    events = [
        MatchEvent(**x).dict()
        for x in df.to_dict(orient="records")
    ]
    
    execution = supabase.table('match_event').upsert(events).execute()
    

In [103]:
supabase = create_client(project_url, project_api)

In [104]:
insert_match_events(df, supabase)

C:\Users\Duy Tran\AppData\Local\Temp\ipykernel_27640\468752302.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  MatchEvent(**x).dict()


In [105]:
team_info = []
team_info.append({
    'team_id': matchdict['home']['teamId'],
    'name': matchdict['home']['name'],
    'country_name': matchdict['home']['countryName'],
    'manager_name': matchdict['home']['managerName'],
    'players': matchdict['home']['players'],
})

team_info.append({
    'team_id': matchdict['away']['teamId'],
    'name': matchdict['away']['name'],
    'country_name': matchdict['away']['countryName'],
    'manager_name': matchdict['away']['managerName'],
    'players': matchdict['away']['players'],
})

In [106]:
matchdict['home']['players']

[{'playerId': 332823,
  'shirtNo': 23,
  'name': 'Unai Simón',
  'position': 'GK',
  'height': 190,
  'weight': 88,
  'age': 27,
  'isFirstEleven': True,
  'isManOfTheMatch': False,
  'field': 'home',
  'stats': {'totalSaves': {'20': 1.0,
    '34': 1.0,
    '55': 1.0,
    '72': 1.0,
    '124': 1.0},
   'collected': {'20': 1.0, '34': 1.0},
   'parriedSafe': {'72': 1.0, '124': 1.0},
   'parriedDanger': {'55': 1.0},
   'claimsHigh': {'26': 1.0, '67': 1.0, '99': 1.0, '134': 1.0},
   'possession': {'0': 1.0,
    '4': 1.0,
    '8': 1.0,
    '10': 1.0,
    '15': 1.0,
    '19': 2.0,
    '20': 1.0,
    '21': 2.0,
    '24': 1.0,
    '26': 1.0,
    '33': 1.0,
    '34': 1.0,
    '36': 2.0,
    '40': 1.0,
    '41': 1.0,
    '43': 1.0,
    '44': 1.0,
    '49': 1.0,
    '51': 1.0,
    '52': 1.0,
    '53': 1.0,
    '57': 1.0,
    '63': 1.0,
    '66': 1.0,
    '67': 1.0,
    '69': 1.0,
    '70': 2.0,
    '71': 1.0,
    '72': 1.0,
    '75': 1.0,
    '78': 1.0,
    '84': 1.0,
    '85': 1.0,
    '89': 1.0

In [107]:
class Player(BaseModel):
    player_id: int
    shirt_no: int
    name: str
    age: int
    position: str
    team_id: int

In [108]:

def insert_players(team_info, supabase):
    players = []
    
    #Loop over each team and create new dictionary for each player format of the table
    for team in team_info:
        for player in team['players']:
            players.append({
                'player_id': player['playerId'],
                'team_id': team['team_id'],
                'shirt_no': player['shirtNo'],
                'name': player['name'],
                'position': player['position'],
                'age': player['age']
            })
                
    execution = supabase.table('player').upsert(players).execute()

In [109]:
insert_players(team_info, supabase)

In [110]:
import psycopg2

conn = psycopg2.connect(
    user ="postgres.srnpjbsmliwxrjfuisfr",
    password="Huytikit321",
    host="aws-0-us-west-1.pooler.supabase.com",
    port=6543,
    database="postgres"
)

In [111]:
cursor = conn.cursor()

In [112]:

cursor.execute("""
    SELECT * FROM player
""")

In [113]:
records = cursor.fetchall()

In [114]:
records

[(30966, 1, 'Fernando Muslera', 38, 'GK', 294),
 (270446, 4, 'Kalvin Phillips', 28, 'Sub', 167),
 (390536, 18, 'Omri Glazer', 28, 'GK', 579),
 (105855, 33, 'Srdjan Mijailovic', 30, 'DR', 579),
 (40560, 15, 'Aleksandar Dragovic', 33, 'DC', 579),
 (450392, 24, 'Nasser Djiga', 21, 'DC', 579),
 (84875, 23, 'Milan Rodic', 33, 'DC', 579),
 (428301, 80, 'Stefan Mitrovic', 22, 'DL', 579),
 (283334, 4, 'Mirko Ivanic', 30, 'MC', 579),
 (443658, 6, 'Marko Stamenic', 22, 'MC', 579),
 (371030, 66, 'Hwang In-Beom', 27, 'MC', 579),
 (401019, 30, 'Osman Bukari', 25, 'FW', 579),
 (400021, 9, 'Cherif Ndiaye', 28, 'FW', 579),
 (318155, 3, 'Milos Degenek', 30, 'Sub', 579),
 (372280, 93, 'Sacha Boey', 23, 'DR', 294),
 (296569, 14, 'Peter Olayinka', 28, 'Sub', 579),
 (444077, 37, 'Vladimir Lucic', 22, 'Sub', 579),
 (378483, 20, 'Kings Kangwa', 25, 'Sub', 579),
 (396585, 17, 'Jean-Philippe Krasso', 27, 'Sub', 579),
 (127429, 8, 'Guélor Kanga', 33, 'Sub', 579),
 (354299, 1, 'Zoran Popovic', 36, 'Sub', 579),
 

In [115]:
#Create a dataframe using the above info
df = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])

In [116]:
df.head(5)

,player_id,shirt_no,name,age,position,team_id
0,30966,1,Fernando Muslera,38,GK,294
1,270446,4,Kalvin Phillips,28,Sub,167
2,390536,18,Omri Glazer,28,GK,579
3,105855,33,Srdjan Mijailovic,30,DR,579
4,40560,15,Aleksandar Dragovic,33,DC,579


# Putting it all together



In [117]:
class MatchEvent(BaseModel):
    id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    player_id: int
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [118]:
def insert_match_events(df, supabase):
    events = [
        MatchEvent(**x).dict()
        for x in df.to_dict(orient="records")
    ]
    
    execution = supabase.table('match_event').upsert(events).execute()

In [119]:
class Player(BaseModel):
    player_id: int
    shirt_no: int
    name: str
    age: int
    position: str
    team_id: int

In [120]:
def insert_players(team_info, supabase):
    players = []
    
    #Loop over each team and create new dictionary for each player format of the table
    for team in team_info:
        for player in team['players']:
            players.append({
                'player_id': player['playerId'],
                'team_id': team['team_id'],
                'shirt_no': player['shirtNo'],
                'name': player['name'],
                'position': player['position'],
                'age': player['age']
            })
                
    execution = supabase.table('player').upsert(players).execute()

In [121]:

supabase = create_client(project_url, project_api)

In [122]:
driver = webdriver.Chrome()

In [123]:
#Create an automation function to 
def scrape_match_events(whoscored_url, driver):
    
    driver.get(whoscored_url)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    element = soup.select_one('script:-soup-contains("matchCentreData")')
    
    matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(',\n')[0])
    
    match_events = matchdict['events']
    
    df = pd.DataFrame(match_events)
    
    df.dropna(subset='playerId', inplace=True)
    
    df = df.where(pd.notnull(df), None)
    
    df = df.rename(
    {
        'eventId': 'event_id',
        'expandedMinute': 'expanded_minute',
        'outcomeType': 'outcome_type',
        'isTouch': 'is_touch',
        'playerId': 'player_id',
        'teamId': 'team_id',
        'endX': 'end_x',
        'endY': 'end_y',
        'blockedX': 'blocked_x',
        'blockedY': 'blocked_y',
        'goalMouthZ': 'goal_mouth_z',
        'goalMouthY': 'goal_mouth_y',
        'isShot': 'is_shot',
        'cardType': 'card_type',
        'isGoal': 'is_goal'
    },
        axis=1
    )
    
    df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
    df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
    df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])
    
    df.drop(columns=["period", "type", "outcome_type"], inplace=True)
    
    if 'is_goal' not in df.columns:
        df['is_goal'] = False
        
    if 'is_card' not in df.columns:
        df['is_card'] = False
        df['card_type'] = False
        
    df = df[~(df['type_display_name'] == "OffsideGiven")]
    
    df = df[[
        'id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
        'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
        'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
        'period_display_name'
    ]]
    
    df[['id', 'event_id', 'minute', 'team_id', 'player_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id']].astype(np.int64)
    df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)
    df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)
    
    df['is_goal'] = df['is_goal'].fillna(False)
    df['is_shot'] = df['is_shot'].fillna(False)
    
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.float32:
            df[column] = np.where(
                np.isnan(df[column]),
                None,
                df[column]
            )
            
            
    insert_match_events(df, supabase)
    
    
    team_info = []
    team_info.append({
        'team_id': matchdict['home']['teamId'],
        'name': matchdict['home']['name'],
        'country_name': matchdict['home']['countryName'],
        'manager_name': matchdict['home']['managerName'],
        'players': matchdict['home']['players'],
    })

    team_info.append({
        'team_id': matchdict['away']['teamId'],
        'name': matchdict['away']['name'],
        'country_name': matchdict['away']['countryName'],
        'manager_name': matchdict['away']['managerName'],
        'players': matchdict['away']['players'],
    })
    
    insert_players(team_info, supabase)
    
    return print('Success')


In [124]:
driver.get('https://www.whoscored.com/Teams/167/Fixtures/England-Manchester-City')
time.sleep(3)

In [125]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [126]:
all_urls = soup.select('a[href*="\/Live\/"]')

In [127]:
#Filter href to get the urls
all_urls = list(set([
    'https://www.whoscored.com' + x.attrs['href']
    for x in all_urls
]))

In [128]:
all_urls

['https://www.whoscored.com/Matches/1789434/Live/Europe-Champions-League-2023-2024-FC-Copenhagen-Manchester-City',
 'https://www.whoscored.com/Matches/1809761/Live/Europe-Champions-League-2023-2024-Real-Madrid-Manchester-City',
 'https://www.whoscored.com/Matches/1775609/Live/Europe-Champions-League-2023-2024-Manchester-City-FK-Crvena-Zvezda',
 'https://www.whoscored.com/Matches/1790091/Live/International-FIFA-Club-World-Cup-2023-Manchester-City-Fluminense',
 'https://www.whoscored.com/Matches/1814147/Live/England-FA-Cup-2023-2024-Manchester-City-Manchester-United',
 'https://www.whoscored.com/Matches/1775611/Live/Europe-Champions-League-2023-2024-BSC-Young-Boys-Manchester-City',
 'https://www.whoscored.com/Matches/1775616/Live/Europe-Champions-League-2023-2024-Manchester-City-RB-Leipzig',
 'https://www.whoscored.com/Matches/1775610/Live/Europe-Champions-League-2023-2024-RB-Leipzig-Manchester-City',
 'https://www.whoscored.com/Matches/1809759/Live/Europe-Champions-League-2023-2024-Manc

In [129]:
for url in all_urls:
    print(url)
    scrape_match_events(
        whoscored_url=url,
        driver=driver
    )
    
    time.sleep(2)

https://www.whoscored.com/Matches/1789434/Live/Europe-Champions-League-2023-2024-FC-Copenhagen-Manchester-City


C:\Users\Duy Tran\AppData\Local\Temp\ipykernel_27640\1487329600.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  MatchEvent(**x).dict()


Success


KeyboardInterrupt: 